In [2]:
import requests

In [225]:
BACKEND_HOST_IP="10.74.4.25"
KONG_URL="http://localhost:8001"
BACKEND_URI="http://10.74.4.25:5000/api"
FRONTEND_URI="http://10.74.4.25:4000/"
FRONTEND_TEST_URI="http://10.74.4.25:4000/test"
AUTH_URI="http://10.65.98.41:8080/"

KONG_CLIENT_ID="kong"
KONG_CLIENT_SECRET="7af12c9d-47b1-4a53-91ac-9acf49b704ef"
REALM_NAME="enki"
SGE_API_KEY = "e86366d6-8ae7-42e5-99b1-cdd04cc9b5b7"

In [226]:
KEYCLOAK_URL = "https://auth.enki.nexsis112.fr"

In [227]:
introspection_url = f'{KEYCLOAK_URL}/auth/realms/{REALM_NAME}/protocol/openid-connect/token/introspect'
discovery_url = f'{KEYCLOAK_URL}/auth/realms/{REALM_NAME}/.well-known/openid-configuration'


In [228]:
discovery_url

'https://auth.enki.nexsis112.fr/auth/realms/enki/.well-known/openid-configuration'

In [229]:
services = [
    {
        'name': "enki_api",
        'url': BACKEND_URI,

        "routes":[
            {
                "path": '/api',
                'auth':'oidc'
            }, 
            {
                'path':'/echanges', 
                'auth':'api_key'
            }
        ]
    },
        {
        'name': "enki_front",
        'url': FRONTEND_URI,
        'routes':[
             {         
                'path': '/',
                "auth":"oidc"
             }, 
        ]
    },
    {
        'name': "enki_front_test",
        'url': FRONTEND_TEST_URI,
        'routes':[
             {         
                'path': '/test',
             }, 
        ]
    }
]

In [233]:
response = requests.get(f'{KONG_URL}/routes')
for _id in [e["id"] for e in response.json()["data"]]:
    requests.delete(f'{KONG_URL}/routes/{_id}')
response = requests.get(f'{KONG_URL}/services')
for _id in [e["id"] for e in response.json()["data"]]:
    requests.delete(f'{KONG_URL}/services/{_id}')

In [234]:
for service in services : 
    print(f"Start configuring {service['name']}")
    response = requests.post(f'{KONG_URL}/services', data={
        'name': service["name"],
        'url': service["url"]
    })
    created_service_id = response.json()["id"]
    for route in service["routes"]:
        auth_type=route.get("auth")

        print(f"configuring route {route['path']}... with {auth_type}")
        response = requests.post(f'{KONG_URL}/services/{service["name"]}/routes', data={
            'service.id': created_service_id,
            'paths[]':route["path"],
        })
        route_id = response.json()["id"]
        
        if auth_type == "api_key":
            response = requests.post(f'{KONG_URL}/routes/{route_id}/plugins', data={
                'name': "key-auth",
                'config.key_names': "x-api-key",
            })
            response = requests.post(f'{KONG_URL}/consumers', data={
                'username': "sge-hub",
            })
            response = requests.post(f'{KONG_URL}/consumers/sge-hub/key-auth', data={
                    'key': apikey,
                })

        elif auth_type == "oidc":
            data = {
                    'name': 'oidc',
                    'config.client_id': f'{KONG_CLIENT_ID}',
                    'config.client_secret': f'{KONG_CLIENT_SECRET}',
                    'config.realm': f'{REALM_NAME}',
                    'config.bearer_only': 'false',
                    'config.introspection_endpoint': introspection_url,
                    'config.discovery': discovery_url
            }

            response = requests.post(f'{KONG_URL}/routes/{route_id}/plugins', data=data)
        else:
            print("No oidc")
            pass

Start configuring enki_api
configuring route /api... with oidc
configuring route /echanges... with api_key
Start configuring enki_front
configuring route /... with oidc
Start configuring enki_front_test
configuring route /test... with None
No oidc


In [214]:
response.json()

{'message': 'Not found'}